# Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import os
import PIL
import pathlib
import tensorflow as tf
from tensorflow import keras
from IPython.display import display
#tf.compat.v1.set_random_seed(290)
# детерминирование случайных величин
tf.random.set_seed(290)
np.random.seed(290)

### Скачивание датасета

In [ ]:
# скачать датасет
# закомменть после того как скачаешь
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True, cache_dir=os.getcwd())
data_dir = pathlib.Path(archive).with_suffix('')

### Разметка данных (изначально в датасете нет размытых изображений)

In [ ]:
TRAIN_PERCENT = 0.60 
VAL_PERCENT = 0.20
TEST_PERCENT = 0.20

IMAGE_WIDTH = 256 # для https://github.com/infernaaa/OIS_FINAL_LABA надо 640на640
IMAGE_HEIGHT = 256

# folder with random photo without blur or labeling
PATH_TO_UNPREPARED_DATASET = 'datasets/flower_photos'
# forler where will be stored labeled data
PATH_TO_SAVE_DATASET = 'labeled_dataset'

dataset = {'train':[],
           'val':[],
           'test':[]}

In [ ]:
# sort all photos to train,val and test datasets (and to sharp and blur ones)
# нужно чтобы представители каждого класса из скачанного датасета присутствовали в 
# каждой выборке
for tup in os.walk(PATH_TO_UNPREPARED_DATASET):
    # tup - это кортеж 3 элементов
    # tup[0] - относительный путь до обрабатываемой папки
    # tup[1] - список всех папок в ней
    # tup[2] - список всех файлов в ней

    # если в папке есть изображения типа jpg
    if [filename for filename in tup[2] if filename.endswith('.jpg')]:
        print(tup[0], tup[1], len(tup[2]), 'items in current path')
        # make every relative path absolute and cast to numpy ndarray
        temp_arr = np.array([pathlib.Path(tup[0])/filename for filename in tup[2]])#[:100]
        # split ndarray of absolute paths to images to 3 parts by percentage
        temp_train, temp_validate, temp_test = np.split(temp_arr, 
                                                        [int(temp_arr.shape[0]*TRAIN_PERCENT), 
                                                         int(temp_arr.shape[0]*(TRAIN_PERCENT+VAL_PERCENT))])
        # add ndarray to list of each dataset part
        # like dataset['train'] = list(np.ndarray(1,2,3), np.ndarray(4,5,6))
        dataset['train'].append(temp_train)
        dataset['val'].append(temp_validate)
        dataset['test'].append(temp_test)

# concatenate each numpy ndarray in each dataset part to one ndarray
for key, value in dataset.items():
    dataset[key] = np.concatenate(value)

In [ ]:
# print dataset info
for key, value in dataset.items():
    print('|'*8,f'{key} part','|'*8)
    print(f'\tTotal items: {value.shape[0]}')
    print(f'\tExample (first 2 items):',*value[:2],sep='\n\t')
    print()

### Обработка размеченных данных (применение размытия) и помощение в новую папку

In [ ]:
from PIL import Image, ImageFilter
# blur labeled images and put all images in new folders
for key, value in dataset.items():
    # create path to save new photo
    path_to_save_blur =  pathlib.Path(PATH_TO_SAVE_DATASET)/key/'blur'
    if not os.path.exists(path_to_save_blur):
        os.makedirs(path_to_save_blur)
        
    path_to_save_sharp =  pathlib.Path(PATH_TO_SAVE_DATASET)/key/'sharp'
    if not os.path.exists(path_to_save_sharp):
        os.makedirs(path_to_save_sharp)

    for image_path in value:
        filename = image_path.name
        
        img = keras.utils.load_img(image_path)
        img = keras.utils.img_to_array(img)
        img = keras.layers.Resizing(IMAGE_HEIGHT, IMAGE_WIDTH)(img)
        img = keras.utils.array_to_img(img)
        img.save(path_to_save_sharp / filename)

        filename = "blur_"+filename
        
        img = img.filter(ImageFilter.BLUR)
        img.save(path_to_save_blur / filename)

### Как добавить имена новых изображение к старому csv файлу из https://github.com/infernaaa/OIS_FINAL_LABA

In [ ]:
'''
# все имена файлов из только что размеченного датасета в датафрейме
# стобцы filenama,dataset,blur
# filename = путь к изображению
# dataset = train, test или val (к какому датасету относится изображение)
# blur = 0 или 1 (заблюрено ли изображение)

dataset_df = pd.DataFrame(data=list(pathlib.Path(PATH_TO_SAVE_DATASET).rglob('*.jpg')), columns=['filename'])
dataset_df['dataset'] = dataset_df['filename'].map(lambda x: x.parent.parent.stem)
dataset_df['blur'] = dataset_df['filename'].map(lambda x: 1 if x.parent.stem=='blur' else 0)

# весь датасет в 1 датафрейме
display(dataset_df)
# только размытые изображения
#display(dataset_df[dataset_df['blur']==1].shape)
# только четкие изображения
#display(dataset_df[dataset_df['blur']==0].shape)''';

In [ ]:
'''
# привести датафрейм к виду как в  https://github.com/infernaaa/OIS_FINAL_LABA с 2 колонками
# train, test и val надо класть в отдельные датафреймы
train_df = dataset_df[dataset_df['dataset']=='train']
train_df = train_df.drop(['dataset'],axis=1)
train_df['filename'] = train_df['filename'].map(lambda x: x.name)

test_df = dataset_df[dataset_df['dataset']=='test']
test_df = test_df.drop(['dataset'],axis=1)
test_df['filename'] = test_df['filename'].map(lambda x: x.name)

display(train_df)
display(test_df)''';

In [ ]:
'''
# прочитать оригинальный csv файл из https://github.com/infernaaa/OIS_FINAL_LABA как датафрейм
path_to_train = pathlib.Path('') # путь как str или pathlib.Path()
path_to_test = pathlib.Path('')
old_train = pd.read_csv(path_to_train)
old_test = pd.read_csv(path_to_train)

# объединить имена новых фото со старыми
new_train = pd.concat([train_df, old_train], axis=0, ignore_index=True)
new_test = pd.concat([test_df, old_test], axis=0, ignore_index=True)

# вывести объединенные датафереймы
display(new_train)
display(new_test)

# сохранить как csv
new_train.to_csv(f'{PATH_TO_SAVE_DATASET}/train/new_train.csv')
new_test.to_csv(f'{PATH_TO_SAVE_DATASET}/test/new_test.csv')

# а вот сами фото в папки в https://github.com/infernaaa/OIS_FINAL_LABA надо ручками перенести
# как и новые файлы csv которыми надо заменить старые 

''';